# Building a Lead Scoring Model with EvalML

In this demo, we will build an optimized lead scoring model using EvalML. To optimize the pipeline, we will set up an objective function to maximize the revenue generated with true positives while taking into account the cost of false positives. At the end of this demo, we also show you how introducing the right objective during the training is over 6x better than using a generic machine learning metric like AUC.

In [1]:
import evalml
from evalml import AutoMLSearch
from evalml.objectives import LeadScoring

## Configure LeadScoring 

To optimize the pipelines toward the specific business needs of this model, you can set your own assumptions for how much value is gained through true positives and the cost associated with false positives. These parameters are

* `true_positive` - dollar amount to be gained with a successful lead
* `false_positive` - dollar amount to be lost with an unsuccessful lead

Using these parameters, EvalML builds a pileline that will maximize the amount of revenue per lead generated.

In [2]:
%%time
lead_scoring_objective = LeadScoring(
    true_positives=1000,
    false_positives=-10
)

CPU times: user 5 µs, sys: 0 ns, total: 5 µs
Wall time: 7.87 µs


## Dataset

We will be utilizing a dataset detailing a customer's job, country, state, zip, online action, the dollar amount of that action and whether they were a successful lead.

In [3]:
%%time
from urllib.request import urlopen
import pandas as pd
import woodwork as ww
customers_data = urlopen('https://featurelabs-static.s3.amazonaws.com/lead_scoring_ml_apps/customers.csv')
interactions_data = urlopen('https://featurelabs-static.s3.amazonaws.com/lead_scoring_ml_apps/interactions.csv')
leads_data = urlopen('https://featurelabs-static.s3.amazonaws.com/lead_scoring_ml_apps/previous_leads.csv')
customers = pd.read_csv(customers_data)
interactions = pd.read_csv(interactions_data)
leads = pd.read_csv(leads_data)

X = customers.merge(interactions, on='customer_id').merge(leads, on='customer_id')
y = X['label']
X = X.drop(['customer_id', 'date_registered', 'birthday','phone', 'email',
            'owner', 'company', 'id', 'time_x',
            'session', 'referrer', 'time_y', 'label', 'country'], axis=1)
display(X.head())

,job,state,zip,action,amount
0,"Engineer, mining",NY,60091.0,page_view,NaN
1,"Psychologist, forensic",CA,NaN,purchase,135.23
2,"Psychologist, forensic",CA,NaN,page_view,NaN
3,Air cabin crew,NaN,60091.0,download,NaN
4,Air cabin crew,NaN,60091.0,page_view,NaN


CPU times: user 88.8 ms, sys: 10.8 ms, total: 99.7 ms
Wall time: 512 ms


We will convert our data into Woodwork data structures. Doing so enables us to have more control over the types passed to and inferred by AutoML.

In [4]:
%%time
X = ww.DataTable(X, semantic_tags={'job': 'category'}, logical_types={'job': 'Categorical'})
y = ww.DataColumn(y)
X.types

CPU times: user 17 ms, sys: 1.44 ms, total: 18.5 ms
Wall time: 17.3 ms


,Physical Type,Logical Type,Semantic Tag(s)
Data Column,,,
job,category,Categorical,['category']
state,category,Categorical,['category']
zip,float64,Double,['numeric']
action,category,Categorical,['category']
amount,float64,Double,['numeric']


## Search for the best pipeline

In order to validate the results of the pipeline creation and optimization process, we will save some of our data as a holdout set.

EvalML natively supports one-hot encoding and imputation so the above `NaN` and categorical values will be taken care of.

In [5]:
%%time
X_train, X_holdout, y_train, y_holdout = evalml.preprocessing.split_data(X, y, problem_type='binary', test_size=0.2, random_state=0)

print(X.types)

            Physical Type Logical Type Semantic Tag(s)
Data Column                                           
job              category  Categorical    ['category']
state            category  Categorical    ['category']
zip               float64       Double     ['numeric']
action           category  Categorical    ['category']
amount            float64       Double     ['numeric']
CPU times: user 10.6 ms, sys: 1.43 ms, total: 12 ms
Wall time: 10.9 ms


Because the lead scoring labels are binary, we will use `AutoMLSearch(X_train=X_train, y_train=y_train, problem_type='binary')`. When we call `.search()`, the search for the best pipeline will begin. 

In [6]:
%%time
automl = AutoMLSearch(X_train=X_train, y_train=y_train,
                      problem_type='binary',
                      objective=lead_scoring_objective,
                      additional_objectives=['auc'],
                      max_batches=1,
                      optimize_thresholds=True)

automl.search()

The following labels fall below 10% of the target: [True]
Generating pipelines to search over...
*****************************
* Beginning pipeline search *
*****************************

Optimizing for Lead Scoring. 
Greater score is better.

Searching up to 1 batches for a total of 9 pipelines. 
Allowed model families: lightgbm, catboost, extra_trees, xgboost, decision_tree, random_forest, linear_model



FigureWidget({
    'data': [{'mode': 'lines+markers',
              'name': 'Best Score',
              'type'…

Batch 1: (1/9) Mode Baseline Binary Classification P... Elapsed:00:00
	Starting cross validation
	Finished cross validation - mean Lead Scoring: 0.000
Batch 1: (2/9) Logistic Regression Classifier w/ Imp... Elapsed:00:01
	Starting cross validation
	Finished cross validation - mean Lead Scoring: 13.784
High coefficient of variation (cv >= 0.2) within cross validation scores. Logistic Regression Classifier w/ Imputer + One Hot Encoder + Standard Scaler may not perform as estimated on unseen data.
Batch 1: (3/9) Random Forest Classifier w/ Imputer +... Elapsed:00:03
	Starting cross validation
	Finished cross validation - mean Lead Scoring: 12.062
High coefficient of variation (cv >= 0.2) within cross validation scores. Random Forest Classifier w/ Imputer + One Hot Encoder may not perform as estimated on unseen data.
Batch 1: (4/9) XGBoost Classifier w/ Imputer + One H... Elapsed:00:05
	Starting cross validation
	Finished cross validation - mean Lead Scoring: 13.173
Batch 1: (5/9) CatBoost

### View rankings and select pipeline

Once the fitting process is done, we can see all of the pipelines that were searched, ranked by their score on the lead scoring objective we defined.

In [7]:
%%time
automl.rankings

CPU times: user 4.1 ms, sys: 316 µs, total: 4.41 ms
Wall time: 4.18 ms


,id,pipeline_name,score,validation_score,percent_better_than_baseline,high_variance_cv,parameters
0,5,Elastic Net Classifier w/ Imputer + One Hot En...,15.997325,12.374194,NaN,False,{'Imputer': {'categorical_impute_strategy': 'm...
1,7,LightGBM Classifier w/ Imputer + One Hot Encoder,14.635258,12.419355,NaN,False,{'Imputer': {'categorical_impute_strategy': 'm...
2,1,Logistic Regression Classifier w/ Imputer + On...,13.784020,9.980645,NaN,True,{'Imputer': {'categorical_impute_strategy': 'm...
3,3,XGBoost Classifier w/ Imputer + One Hot Encoder,13.172704,11.316129,NaN,False,{'Imputer': {'categorical_impute_strategy': 'm...
4,6,Extra Trees Classifier w/ Imputer + One Hot En...,13.028294,12.496774,NaN,False,{'Imputer': {'categorical_impute_strategy': 'm...
5,4,CatBoost Classifier w/ Imputer,12.640320,12.374194,NaN,True,{'Imputer': {'categorical_impute_strategy': 'm...
6,2,Random Forest Classifier w/ Imputer + One Hot ...,12.062318,11.212903,NaN,True,{'Imputer': {'categorical_impute_strategy': 'm...
7,8,Decision Tree Classifier w/ Imputer + One Hot ...,11.531380,10.161290,NaN,False,{'Imputer': {'categorical_impute_strategy': 'm...
8,0,Mode Baseline Binary Classification Pipeline,0.000000,0.000000,NaN,False,{'Baseline Classifier': {'strategy': 'mode'}}


To select the best pipeline we can call `automl.best_pipeline`.

In [8]:
%%time
best_pipeline = automl.best_pipeline

CPU times: user 4 µs, sys: 1 µs, total: 5 µs
Wall time: 6.91 µs


### Describe pipeline

You can get more details about any pipeline, including how it performed on other objective functions by calling `.describe_pipeline()` and specifying the `id` of the pipeline.

In [9]:
%%time
automl.describe_pipeline(automl.rankings.iloc[0]["id"])

*************************************************************************
* Elastic Net Classifier w/ Imputer + One Hot Encoder + Standard Scaler *
*************************************************************************

Problem Type: binary
Model Family: Linear

Pipeline Steps
1. Imputer
	 * categorical_impute_strategy : most_frequent
	 * numeric_impute_strategy : mean
	 * categorical_fill_value : None
	 * numeric_fill_value : None
2. One Hot Encoder
	 * top_n : 10
	 * features_to_encode : None
	 * categories : None
	 * drop : None
	 * handle_unknown : ignore
	 * handle_missing : error
3. Standard Scaler
4. Elastic Net Classifier
	 * alpha : 0.5
	 * l1_ratio : 0.5
	 * n_jobs : -1
	 * max_iter : 1000
	 * penalty : elasticnet
	 * loss : log

Training
Training for binary problems.
Objective to optimize binary classification pipeline thresholds for: <evalml.objectives.lead_scoring.LeadScoring object at 0x1055c6e20>
Total training time (including CV): 1.2 seconds

Cross Validation
------

## Evaluate on hold out

Finally, since the best pipeline was trained on all of the training data, we evaluate it on the holdout dataset.

In [10]:
%%time
best_pipeline.score(X_holdout, y_holdout, objectives=["auc", lead_scoring_objective])

CPU times: user 398 ms, sys: 148 ms, total: 546 ms
Wall time: 117 ms


OrderedDict([('AUC', 0.5), ('Lead Scoring', 15.382631126397248)])

## Why optimize for a problem-specific objective?

To demonstrate the importance of optimizing for the right objective, let's search for another pipeline using AUC, a common machine learning metric. After that, we will score the holdout data using the lead scoring objective to see how the best pipelines compare.

In [11]:
%%time
automl_auc = evalml.AutoMLSearch(X_train=X_train, y_train=y_train,
                                 problem_type='binary',
                                 objective='auc',
                                 additional_objectives=[],
                                 max_batches=1,
                                 optimize_thresholds=True)

automl_auc.search()

The following labels fall below 10% of the target: [True]
Generating pipelines to search over...
*****************************
* Beginning pipeline search *
*****************************

Optimizing for AUC. 
Greater score is better.

Searching up to 1 batches for a total of 9 pipelines. 
Allowed model families: lightgbm, catboost, extra_trees, xgboost, decision_tree, random_forest, linear_model



FigureWidget({
    'data': [{'mode': 'lines+markers',
              'name': 'Best Score',
              'type'…

Batch 1: (1/9) Mode Baseline Binary Classification P... Elapsed:00:00
	Starting cross validation
	Finished cross validation - mean AUC: 0.500
Batch 1: (2/9) Logistic Regression Classifier w/ Imp... Elapsed:00:00
	Starting cross validation
	Finished cross validation - mean AUC: 0.695
Batch 1: (3/9) Random Forest Classifier w/ Imputer +... Elapsed:00:01
	Starting cross validation
	Finished cross validation - mean AUC: 0.702
Batch 1: (4/9) XGBoost Classifier w/ Imputer + One H... Elapsed:00:02
	Starting cross validation
	Finished cross validation - mean AUC: 0.709
Batch 1: (5/9) CatBoost Classifier w/ Imputer           Elapsed:00:02
	Starting cross validation
	Finished cross validation - mean AUC: 0.557
Batch 1: (6/9) Elastic Net Classifier w/ Imputer + O... Elapsed:00:03
	Starting cross validation
	Finished cross validation - mean AUC: 0.500
Batch 1: (7/9) Extra Trees Classifier w/ Imputer + O... Elapsed:00:03
	Starting cross validation
	Finished cross validation - mean AUC: 0.722
Batch 

In [12]:
%%time
automl_auc.rankings

CPU times: user 3.99 ms, sys: 277 µs, total: 4.26 ms
Wall time: 4.15 ms


,id,pipeline_name,score,validation_score,percent_better_than_baseline,high_variance_cv,parameters
0,6,Extra Trees Classifier w/ Imputer + One Hot En...,0.722008,0.718159,44.401691,False,{'Imputer': {'categorical_impute_strategy': 'm...
1,3,XGBoost Classifier w/ Imputer + One Hot Encoder,0.708940,0.696173,41.788046,False,{'Imputer': {'categorical_impute_strategy': 'm...
2,2,Random Forest Classifier w/ Imputer + One Hot ...,0.701576,0.699430,40.315221,False,{'Imputer': {'categorical_impute_strategy': 'm...
3,1,Logistic Regression Classifier w/ Imputer + On...,0.694583,0.685778,38.916659,False,{'Imputer': {'categorical_impute_strategy': 'm...
4,7,LightGBM Classifier w/ Imputer + One Hot Encoder,0.678321,0.668776,35.664140,False,{'Imputer': {'categorical_impute_strategy': 'm...
5,8,Decision Tree Classifier w/ Imputer + One Hot ...,0.586996,0.560719,17.399261,False,{'Imputer': {'categorical_impute_strategy': 'm...
6,4,CatBoost Classifier w/ Imputer,0.556956,0.581441,11.391175,False,{'Imputer': {'categorical_impute_strategy': 'm...
7,0,Mode Baseline Binary Classification Pipeline,0.500000,0.500000,0.000000,False,{'Baseline Classifier': {'strategy': 'mode'}}
8,5,Elastic Net Classifier w/ Imputer + One Hot En...,0.500000,0.500000,0.000000,False,{'Imputer': {'categorical_impute_strategy': 'm...


Like before, we can look at the rankings and pick the best pipeline.

In [13]:
%%time
best_pipeline_auc = automl_auc.best_pipeline

CPU times: user 5 µs, sys: 0 ns, total: 5 µs
Wall time: 8.11 µs


In [14]:
%%time
# get the auc and lead scoring score on holdout data
best_pipeline_auc.score(X_holdout, y_holdout,  objectives=["auc", lead_scoring_objective])

CPU times: user 130 ms, sys: 16.1 ms, total: 146 ms
Wall time: 129 ms


OrderedDict([('AUC', 0.6735040797824116),
             ('Lead Scoring', -0.008598452278589854)])

When we optimize for AUC, we can see that the AUC score from this pipeline is better than the AUC score from the pipeline optimized for lead scoring. However, the revenue per lead gained was only `$7` per lead when optimized for AUC and was `$45` when optimized for lead scoring. As a result, we would gain up to 6x the amount of revenue if we optimized for lead scoring.

This happens because optimizing for AUC does not take into account the user-specified true_positive (dollar amount to be gained with a successful lead) and false_positive (dollar amount to be lost with an unsuccessful lead) values. Thus, the best pipelines may produce the highest AUC but may not actually generate the most revenue through lead scoring.

This example highlights how performance in the real world can diverge greatly from machine learning metrics.